In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/phishing-data/combined_dataset.csv
/kaggle/input/phishing/Phishing_Legitimate_full.csv


## Data collection and Pre-Process

In [2]:
!pip install --upgrade tensorflow-federated
import tensorflow_federated as tff
!pip install tensorflow_federated
from tensorflow_federated import python as tff



/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ERROR: Could not find a version that satisfies the requirement tensorflow-federated (from versions: none)
ERROR: No matching distribution found for tensorflow-federated


ModuleNotFoundError: No module named 'tensorflow_federated'

In [ ]:
 # create and train federated learning models
    
import tensorflow as tf
import tensorflow_federated as tff

# Define the client data loading function
def load_data_for_client(client_id):
  # Load the data for the specified client
  data = load_data_from_file(client_id + 'phishing.csv')

  # Preprocess the data
  data = preprocess_data(data)

  # Split the data into features and labels
  features = data[:, :-1]
  labels = data[:, -1]

  # Create a TensorFlow dataset from the features and labels
  dataset = tf.data.Dataset.from_tensor_slices({'x': features, 'y': labels})

  return dataset

In [ ]:
# Define the federated data loading function
def load_data_for_federated_learning(client_ids):
  # Load the data for each client and create a federated dataset
  federated_data = tff.simulation.datasets.ClientData.from_clients_and_fn(
      client_ids=client_ids,
      create_tf_dataset_for_client_fn=lambda client_id: load_data_for_client(client_id)
  )

  return federated_data

In [ ]:
# Define the preprocessing function
def preprocess_data(data):
  # Perform data preprocessing steps, such as normalization, feature engineering, etc.
  return data



In [ ]:
# Define the federated data preprocessing function
def preprocess_data_for_federated_learning(client_ids):
  # Load the data for each client and perform preprocessing steps
  preprocessed_data = {}
  for client_id in client_ids:
    data = load_data_from_file(client_id + 'phishing-data.csv')
    preprocessed_data[client_id] = preprocess_data(data)

  return preprocessed_data

## Model Training

In [ ]:
import tensorflow as tf
from tensorflow_federated import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

In [ ]:
# Define the model
# allows you to build a deep learning model by simply adding one layer at a time.
model = Sequential([
  Dense(32, activation='relu', input_shape=(input_size,)),
  Dense(1, activation='sigmoid')
])


In [ ]:
# Compile the model
# binary cross entropy difference between the predicted output of a neural network and the true output.
# optimization algorithm for training neural networks
model.compile(
  loss='binary_crossentropy',
  optimizer=optimizers.SGD(learning_rate=0.01),
  metrics=['accuracy']
)


In [ ]:
# Convert the model to a federated model
federated_model = keras.models.clone_model(model)
federated_model = keras.layers.TimeDistributed(federated_model)

In [ ]:
# Define the training algorithm
# TrainOneEpoch defines the training algorithm 
trainer = keras.TrainOneEpoch(model=federated_model, optimizer=optimizers.SGD(learning_rate=0.01))

In [ ]:

# Train the model on the federated data
federated_data = preprocess_data(data) # Preprocessed data
for i in range(num_epochs):
  federated_model = trainer(federated_model, federated_data)

## MOdel Testing

In [ ]:
# Evaluate the model on a separate dataset
test_data = preprocess_data(test_data) # Preprocessed test data
loss, accuracy = model.evaluate(test_data)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')
